
## Задача №6

Пекарня пече хліб на продаж магазинам. Собівартість однієї булки становить 30 пенсів, її продають за 40 пенсів. У таблиці наведено дані про попит за останні 50 днів:
Попит на день, тис. шт.	10	12	14	16	18
Число днів	5	10	15	15	5
Якщо булка спечена, але не продана, то збитки складуть 20 пенсів за штуку. Визначте, скільки булок потрібно випікати в день.



### Вводимо початкові дані

In [35]:

demand = [10, 12, 14, 16, 18] 
amountOfDays = [5, 10, 15, 15, 5] 

profit = 40 - 30 
losses_non_sale = 20 

In [36]:
import numpy as np
import pandas as pd


## Рахуємо ймовірність

In [37]:
probability_demand = amountOfDays / np.sum(amountOfDays) 

print("Ймовірність попиту на період: ", probability_demand)

Ймовірність попиту на період:  [0.1 0.2 0.3 0.3 0.1]


### Побудова дерева прийняття рішень

In [38]:
def calcDecisionTree(_tree_results, _tree_income_results):
    for i in range(len(demand)):
        temp = []
        fullExpectedIncome = 0
        fullExpectedExtIncome = 0
        for j in range(len(demand)):
            soldPerDay = min(demand[i], demand[j])
            notSoldPerDay = max(0, demand[i] - demand[j])
            expectedIncome = profit * soldPerDay
            expectedIncomeExt = expectedIncome - losses_non_sale * notSoldPerDay
            fullExpectedIncome += expectedIncome * probability_demand[j]
            fullExpectedExtIncome += expectedIncomeExt * probability_demand[j]
            temp.append([
                probability_demand[j],
                demand[i],
                demand[j],
                soldPerDay,
                notSoldPerDay,
                max(0, demand[j] - demand[i]),
                expectedIncome,
                expectedIncomeExt,
            ])
        _tree_results.append(temp)
        _tree_income_results.append([fullExpectedIncome, fullExpectedExtIncome])

## Обраховуємо дерево прийняття рішень

In [39]:
tree_results = []
tree_income_results = []

calcDecisionTree(tree_results, tree_income_results)

In [40]:
for i in range(len(tree_results)):
	print(pd.DataFrame(
        tree_results[i],
        columns=[
            "Ймовірність попиту",
            "Закупка",
            "Запит",
            "Продан",
            "Не продано",
            "Незадоволений попит",
            "Очікуваний чистий дохід",
            "дохід з урахуванням втрат",
		]
	))

   Ймовірність попиту  Закупка  Запит  Продан  Не продано  \
0                 0.1       10     10      10           0   
1                 0.2       10     12      10           0   
2                 0.3       10     14      10           0   
3                 0.3       10     16      10           0   
4                 0.1       10     18      10           0   

   Незадоволений попит  Очікуваний чистий дохід  дохід з урахуванням втрат  
0                    0                      100                        100  
1                    2                      100                        100  
2                    4                      100                        100  
3                    6                      100                        100  
4                    8                      100                        100  
   Ймовірність попиту  Закупка  Запит  Продан  Не продано  \
0                 0.1       12     10      10           2   
1                 0.2       12     12      12    

In [41]:
print(pd.DataFrame(
    tree_income_results,
    columns=[
            "чистий дохід",
            "дохід з урахуванням витрат",
		]
))

   чистий дохід  дохід з урахуванням витрат
0         100.0                       100.0
1         118.0                       114.0
2         132.0                       116.0
3         140.0                       100.0
4         142.0                        66.0


### визначення оптимального рішення

In [42]:
maxExpectedIncome = np.max(tree_income_results, axis=0)[0]
alternative = np.where(tree_income_results == maxExpectedIncome)[0][0]

print(
	"Очікуваний чистий дохід максимальний: "
	"альтернатива " + str(alternative + 1) +
	". Дохід: " + str(maxExpectedIncome) +" пенсів."
)

maxExpectedIncome = np.max(tree_income_results, axis=0)[1]
alternative = np.where(tree_income_results == maxExpectedIncome)[0][0]

print(
	"Максимальний дохід з урахуванням втрат: "
	"альтернатива " + str(alternative + 1) +
	". Дохід: " + str(maxExpectedIncome) +" пенсів."
)

Очікуваний чистий дохід максимальний: альтернатива 5. Дохід: 142.0 пенсів.
Максимальний дохід з урахуванням втрат: альтернатива 3. Дохід: 116.0 пенсів.


### Аналіз чутливості прийняття рішень

**Перевіримо чи залежить вибір від значень ймовірностей**

In [43]:
probability_demand = [0.1, 0.1, 0.5, 0.2, 0.1]
test1 = []
calcDecisionTree([], test1)

probability_demand = [0.4, 0.1, 0.1, 0.3, 0.1]
test2 = []
calcDecisionTree([], test2)

probability_demand = [0.2, 0.2, 0.2, 0.2, 0.2]
test3 = []
calcDecisionTree([], test3)

print(test1)
print(test2)
print(test3)

[[100.0, 100.0], [118.0, 114.0], [134.0, 122.0], [140.0, 100.0], [142.0, 66.0]]
[[100.0, 100.0], [112.0, 96.0], [122.0, 86.0], [130.0, 70.0], [132.0, 36.0]]
[[100.0, 100.0], [116.0, 108.0], [128.0, 104.0], [136.0, 88.0], [140.0, 60.0]]


**З результатів помітно, що при зміні ймовірностей, оптимальне рішення теж змінюється.**